In [2]:
from pathlib import Path

PROJECT_ROOT = Path(r"C:\Users\serrios\01_prueba_analitico_4")
DB_PATH = PROJECT_ROOT / "database" / "analytics.duckdb"

print("PROJECT_ROOT:", PROJECT_ROOT)
print("DB_PATH:", DB_PATH)
print("Existe la carpeta 'database'? ->", (PROJECT_ROOT / "database").exists())
print("Existe el archivo de DuckDB?  ->", DB_PATH.exists())

PROJECT_ROOT: C:\Users\serrios\01_prueba_analitico_4
DB_PATH: C:\Users\serrios\01_prueba_analitico_4\database\analytics.duckdb
Existe la carpeta 'database'? -> False
Existe el archivo de DuckDB?  -> False


In [3]:
from pathlib import Path
import duckdb

# 1) Rutas
PROJECT_ROOT = Path(r"C:\Users\serrios\01_prueba_analitico_4")
DB_DIR = PROJECT_ROOT / "database"
DB_PATH = DB_DIR / "analytics.duckdb"

# 2) Crear carpeta si no existe
DB_DIR.mkdir(parents=True, exist_ok=True)
print("Carpeta creada/existente:", DB_DIR)

# 3) Crear/abrir la base DuckDB (si no existe, la crea)
con = duckdb.connect(str(DB_PATH))

# 4) Crear el schema 'gold' (idempotente)
con.execute("CREATE SCHEMA IF NOT EXISTS gold;")

# 5) (Opcional, solo para validar)
schemas = con.execute("SELECT schema_name FROM information_schema.schemata").fetchdf()
print("Schemas existentes:\n", schemas)

con.close()
print("Base creada en:", DB_PATH)

Carpeta creada/existente: C:\Users\serrios\01_prueba_analitico_4\database
Schemas existentes:
           schema_name
0                gold
1  information_schema
2                main
3          pg_catalog
4  information_schema
5                main
6          pg_catalog
7  information_schema
8                main
9          pg_catalog
Base creada en: C:\Users\serrios\01_prueba_analitico_4\database\analytics.duckdb


In [5]:
import duckdb
from pathlib import Path

DB_PATH = Path(r"C:\Users\serrios\01_prueba_analitico_4\database\analytics.duckdb")
con = duckdb.connect(str(DB_PATH))

print("Objetos 'dataset_%' (tablas/vistas):")
print(con.execute("""
    SELECT 
        table_schema, 
        table_name, 
        table_type -- 'BASE TABLE' o 'VIEW'
    FROM information_schema.tables
    WHERE table_name ILIKE 'dataset_%'
    ORDER BY table_schema, table_name
""").fetchdf())

con.close()

Objetos 'dataset_%' (tablas/vistas):
Empty DataFrame
Columns: [table_schema, table_name, table_type]
Index: []


In [7]:
from pathlib import Path

root = Path(r"C:\Users\serrios")  # puedes acotar si sabes otra carpeta base
matches = list(root.rglob("analytics.duckdb"))
print("Coincidencias encontradas:")
for p in matches:
    print("-", p)


Coincidencias encontradas:
- C:\Users\serrios\01_prueba_analitico_4\database\analytics.duckdb
- C:\Users\serrios\01_prueba_analitico_4\op_cobro\database\analytics.duckdb


In [8]:
from pathlib import Path

# Ruta al proyecto base (ajústala si tu repo está en otra carpeta raíz)
PROJECT_ROOT = Path(r"C:\Users\serrios\01_prueba_analitico_4")

# 1) Base DuckDB destino (la que usará tu script de entrenamiento)
DB_DIR = PROJECT_ROOT / "database"
DB_PATH = DB_DIR / "analytics.duckdb"

# 2) Ruta al dataset GOLD en Parquet (la que mostraste en la imagen)
GOLD_PARQUET_DIR = PROJECT_ROOT / "op_cobro" / "database" / "staging" / "gold.dataset_modelo.parquet_write"

print("DB_PATH:", DB_PATH)
print("GOLD_PARQUET_DIR:", GOLD_PARQUET_DIR)

DB_PATH: C:\Users\serrios\01_prueba_analitico_4\database\analytics.duckdb
GOLD_PARQUET_DIR: C:\Users\serrios\01_prueba_analitico_4\op_cobro\database\staging\gold.dataset_modelo.parquet_write


In [9]:
import duckdb
from pathlib import Path

DB_DIR.mkdir(parents=True, exist_ok=True)

con = duckdb.connect(str(DB_PATH))
con.execute("CREATE SCHEMA IF NOT EXISTS gold;")
con.close()

print("✅ Base creada/abierta y schema 'gold' asegurado:", DB_PATH)

✅ Base creada/abierta y schema 'gold' asegurado: C:\Users\serrios\01_prueba_analitico_4\database\analytics.duckdb


In [10]:
import duckdb

# Validar que la carpeta existe y que hay archivos parquet
assert GOLD_PARQUET_DIR.exists(), f"No existe la carpeta {GOLD_PARQUET_DIR}"
parts = list(GOLD_PARQUET_DIR.glob("*.parquet"))
assert parts, f"No se encontraron archivos .parquet en {GOLD_PARQUET_DIR}"

# Crear tabla GOLD a partir de los parquet
con = duckdb.connect(str(DB_PATH))
con.execute("CREATE SCHEMA IF NOT EXISTS gold;")

# OJO: usa .as_posix() para que DuckDB acepte los separadores en Windows
parquet_glob = (GOLD_PARQUET_DIR / "part-*.parquet").as_posix()

con.execute(f"""
    CREATE OR REPLACE TABLE gold.dataset_modelo AS
    SELECT * FROM read_parquet('{parquet_glob}');
""")

# (Opcional) Vista alias que espera tu script
con.execute("""
    CREATE OR REPLACE VIEW gold.dataset_final AS
    SELECT * FROM gold.dataset_modelo;
""")

# Validación rápida: columnas existentes
print(con.execute("DESCRIBE gold.dataset_final").fetchdf().head(20))

con.close()
print("✅ Tabla gold.dataset_modelo creada desde Parquet y vista gold.dataset_final lista.")

              column_name column_type null   key default extra
0             num_doc_key     VARCHAR  YES  None    None  None
1               obl17_key     VARCHAR  YES  None    None  None
2              f_analisis        DATE  YES  None    None  None
3                 num_doc      BIGINT  YES  None    None  None
4                   obl17      BIGINT  YES  None    None  None
5                 var_rta      BIGINT  YES  None    None  None
6       moras_avg_mora_3m      DOUBLE  YES  None    None  None
7       moras_min_mora_3m      BIGINT  YES  None    None  None
8       moras_max_mora_3m      BIGINT  YES  None    None  None
9    moras_stddev_mora_3m      DOUBLE  YES  None    None  None
10      moras_avg_mora_6m      DOUBLE  YES  None    None  None
11      moras_min_mora_6m      BIGINT  YES  None    None  None
12      moras_max_mora_6m      BIGINT  YES  None    None  None
13   moras_stddev_mora_6m      DOUBLE  YES  None    None  None
14      moras_avg_mora_9m      DOUBLE  YES  None    Non

In [12]:
import duckdb
from pathlib import Path

PROJECT_ROOT = Path(r"C:\Users\serrios\01_prueba_analitico_4")
DB_PATH = PROJECT_ROOT / "database" / "analytics.duckdb"
GOLD_PARQUET_DIR = PROJECT_ROOT / "op_cobro" / "database" / "staging" / "gold.dataset_modelo.parquet_write"

parquet_glob = (GOLD_PARQUET_DIR / "part-*.parquet").as_posix()

con = duckdb.connect(str(DB_PATH))

print("👀 Columnas detectadas en Parquet:")
print(con.execute(f"""
    SELECT *
    FROM read_parquet('{parquet_glob}')
    LIMIT 0
""").fetchdf().columns.tolist())

print("\n🔎 10 filas de muestra (solo columnas candidatas):")
# Trae un subconjunto común; ajusta si no existen
print(con.execute(f"""
    SELECT 
        * 
    FROM read_parquet('{parquet_glob}')
    LIMIT 10
""").fetchdf())

con.close()


👀 Columnas detectadas en Parquet:
['num_doc_key', 'obl17_key', 'f_analisis', 'num_doc', 'obl17', 'var_rta', 'moras_avg_mora_3m', 'moras_min_mora_3m', 'moras_max_mora_3m', 'moras_stddev_mora_3m', 'moras_avg_mora_6m', 'moras_min_mora_6m', 'moras_max_mora_6m', 'moras_stddev_mora_6m', 'moras_avg_mora_9m', 'avg_pago_debito_3m', 'min_pago_debito_3m', 'max_pago_debito_3m', 'stddev_pago_debito_3m', 'avg_pago_fisico_3m', 'min_pago_fisico_3m', 'max_pago_fisico_3m', 'stddev_pago_fisico_3m', 'avg_pago_virtual_3m', 'min_pago_virtual_3m', 'max_pago_virtual_3m', 'stddev_pago_virtual_3m', 'avg_pago_otros_3m', 'min_pago_otros_3m', 'max_pago_otros_3m', 'stddev_pago_otros_3m', 'avg_pago_debito_6m', 'min_pago_debito_6m', 'max_pago_debito_6m', 'stddev_pago_debito_6m', 'avg_pago_fisico_6m', 'min_pago_fisico_6m', 'max_pago_fisico_6m', 'stddev_pago_fisico_6m', 'avg_pago_virtual_6m', 'min_pago_virtual_6m', 'max_pago_virtual_6m', 'stddev_pago_virtual_6m', 'avg_pago_otros_6m', 'min_pago_otros_6m', 'max_pago_otro